Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'

REPO_ROOT = "/home/leon/models/NeuroFlame"
pal = sns.color_palette("tab10")
```

Imports
=======

``` ipython
import lightning as L
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader

DEVICE = 'cuda:1'
```

``` ipython
import sys
sys.path.insert(0, '../')

import pandas as pd
import torch.nn as nn
from time import perf_counter
from scipy.stats import circmean

from src.network import Network
from src.lrnet import LRNet

from src.plot_utils import plot_con
from src.decode import decode_bump, circcvl
from src.lr_utils import masked_normalize, clamp_tensor, normalize_tensor
```

Helpers
=======

Data Split
----------

``` ipython
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

def split_data(X, Y, train_perc=0.8, batch_size=32):

  if Y.ndim==3:
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                        train_size=train_perc,
                                                        stratify=Y[:, 0, 0].cpu().numpy(),
                                                        shuffle=True)
  else:
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                        train_size=train_perc,
                                                        stratify=Y[:, 0].cpu().numpy(),
                                                        shuffle=True)
  print(X_train.shape, X_test.shape)
  print(Y_train.shape, Y_test.shape)

  train_dataset = TensorDataset(X_train, Y_train)
  val_dataset = TensorDataset(X_test, Y_test)

  # Create data loaders
  train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, val_loader
```

Configuration
=============

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"
conf_name = "config_light.yml"
DEVICE = 'cuda:1'
seed = np.random.randint(0, 1e6)
print(seed)
#seed = 760946
```

Model
=====

``` ipython
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE=DEVICE, SEED=seed, N_BATCH=16)
```

Dataset
=======

``` ipython
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)
```

``` ipython
model.LR_TRAIN = 1
model.LR_READOUT=1
```

Testing the network on steps from sample odor offset to test odor onset

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)

mask = (steps >= (model.N_STIM_OFF[0] - model.N_STEADY)) & (steps <= (model.N_STEPS - model.N_STEADY))
rwd_idx = np.where(mask)[0]
print('rwd', rwd_idx)

model.lr_eval_win = rwd_idx.shape[0]

stim_mask = (steps >= (model.N_STIM_ON[0] - model.N_STEADY)) & (steps < (model.N_STIM_OFF[0] - model.N_STEADY))

zero_idx = np.where(~mask & ~stim_mask )[0]
print('zero', zero_idx)
```

### Inputs and Labels

``` ipython
model.N_BATCH = 64

model.I0[0] = 2.0
model.I0[1] = 0
model.I0[2] = 0

A = model.init_ff_input()

model.I0[0] = -2.0
model.I0[1] = 0
model.I0[2] = 0

B = model.init_ff_input()

ff_input = torch.cat((A, B))
print(ff_input.shape)
```

``` ipython
labels_A = torch.ones((model.N_BATCH, 1))
labels_B = torch.zeros((model.N_BATCH, 1))
labels = torch.cat((labels_A, labels_B))

print('labels', labels.shape)
```

``` ipython
batch_size = 16
train_loader, val_loader = split_data(ff_input.to(DEVICE), labels.to(DEVICE), train_perc=0.8, batch_size=batch_size)
```

Run
===

``` ipython
autoencoder = LRNet(model)
```

``` ipython
y_pred = autoencoder()
```

``` ipython
print(y_pred.shape)
```

``` ipython
from lightning.pytorch.callbacks import ModelCheckpoint

# Init ModelCheckpoint callback, monitoring 'val_loss'
checkpoint_callback = ModelCheckpoint(monitor="val_loss")

# Add your callback to the callbacks list
```

``` ipython
trainer = L.Trainer(devices=[1], max_epochs=30, num_sanity_val_steps=0, callbacks=[checkpoint_callback], enable_progress_bar=0)
```

``` ipython
trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=val_loader);
```

``` example
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | Network           | 2.5 K
1 | linear    | Linear            | 501
2 | criterion | BCEWithLogitsLoss | 0
------------------------------------------------
3.0 K     Trainable params
0         Non-trainable params
3.0 K     Total params
0.012     Total estimated model params size (MB)
Epoch 0 - Training loss: 0.20096835494041443 - Validation loss: 0.12238327413797379
```

``` ipython
train_loss = trainer.logged_metrics.get('train_loss')
val_loss = trainer.logged_metrics.get('val_loss')

print(f'Training Loss: {train_loss}, Validation Loss: {val_loss}')
```